In [51]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv
/kaggle/input/playground-series-s3e2/sample_submission.csv
/kaggle/input/playground-series-s3e2/train.csv
/kaggle/input/playground-series-s3e2/test.csv


In [52]:
sd=pd.read_csv("/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv")
sd

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [53]:
mu_bmi = round(sd['bmi'].mean(), ndigits=1)
print (mu_bmi)
sd['bmi'].fillna(mu_bmi,inplace = True)
#sd = sd.dropna()

28.9


In [54]:
sd

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.9,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,28.9,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [55]:
sda=pd.read_csv("/kaggle/input/playground-series-s3e2/train.csv")
sda.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [56]:
sda

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0
...,...,...,...,...,...,...,...,...,...,...,...,...
15299,15299,Female,22.0,0,0,No,Govt_job,Urban,72.63,19.5,never smoked,0
15300,15300,Female,46.0,1,0,Yes,Private,Urban,101.19,32.1,never smoked,0
15301,15301,Female,75.0,0,0,Yes,Self-employed,Urban,87.69,26.2,never smoked,0
15302,15302,Male,46.0,0,0,Yes,Private,Rural,101.13,22.5,Unknown,0


In [57]:
sda = sda.reset_index(drop=True)
ft = sd.append(sda)
ft = ft.reset_index(drop=True)
ft.drop('id', axis=1, inplace=True)
#ft = ft.sample(frac = 1)
ft

/tmp/ipykernel_34/2524880480.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ft = sd.append(sda)


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.9,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
20409,Female,22.0,0,0,No,Govt_job,Urban,72.63,19.5,never smoked,0
20410,Female,46.0,1,0,Yes,Private,Urban,101.19,32.1,never smoked,0
20411,Female,75.0,0,0,Yes,Self-employed,Urban,87.69,26.2,never smoked,0
20412,Male,46.0,0,0,Yes,Private,Rural,101.13,22.5,Unknown,0


In [58]:
ft = ft.sample(frac = 1)
ft.reset_index(drop = True, inplace = True)
ft

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Female,24.0,0,0,No,Private,Rural,102.91,27.4,formerly smoked,0
1,Female,17.0,0,0,No,Private,Urban,96.47,25.6,Unknown,0
2,Female,59.0,0,0,Yes,Private,Rural,70.88,29.4,formerly smoked,0
3,Male,42.0,0,0,Yes,Private,Rural,107.83,35.3,smokes,0
4,Male,26.0,0,0,Yes,Govt_job,Rural,73.29,27.8,smokes,0
...,...,...,...,...,...,...,...,...,...,...,...
20409,Male,17.0,0,0,No,Private,Rural,79.61,24.5,smokes,0
20410,Female,30.0,0,0,Yes,Private,Urban,79.35,30.3,never smoked,0
20411,Female,61.0,1,0,Yes,Private,Urban,77.55,29.7,Unknown,0
20412,Female,31.0,0,0,No,Private,Urban,99.44,35.5,never smoked,0


In [59]:
ft.isna().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [60]:
work_mapping = {'children': 0, 'Govt_job': 1, 'Never_worked': 2, 'Private': 3, 'Self-employed': 3}
gender_mapping = {'Male': 0, 'Female': 1, 'Other': 2}
marriage_mapping = {'No': 0, 'Yes': 1}
residence_mapping = {'Rural': 0, 'Urban': 1}
smoke_mapping = {'formerly smoked': 0, 'never smoked': 1, 'smokes': 2, 'Unknown': 3}
ft['work_type'] = ft['work_type'].replace(work_mapping)
ft['gender'] = ft['gender'].replace(gender_mapping)
ft['ever_married'] = ft['ever_married'].replace(marriage_mapping)
ft['Residence_type'] = ft['Residence_type'].replace(residence_mapping)
ft['smoking_status'] = ft['smoking_status'].replace(smoke_mapping)
ft

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,24.0,0,0,0,3,0,102.91,27.4,0,0
1,1,17.0,0,0,0,3,1,96.47,25.6,3,0
2,1,59.0,0,0,1,3,0,70.88,29.4,0,0
3,0,42.0,0,0,1,3,0,107.83,35.3,2,0
4,0,26.0,0,0,1,1,0,73.29,27.8,2,0
...,...,...,...,...,...,...,...,...,...,...,...
20409,0,17.0,0,0,0,3,0,79.61,24.5,2,0
20410,1,30.0,0,0,1,3,1,79.35,30.3,1,0
20411,1,61.0,1,0,1,3,1,77.55,29.7,3,0
20412,1,31.0,0,0,0,3,1,99.44,35.5,1,0


In [61]:
X = ft.drop('stroke', axis=1)
y = ft['stroke']

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [64]:
model = RandomForestClassifier()
model.fit(X_train, y_train)


RandomForestClassifier()

In [65]:
accuracy_val = model.score(X_val, y_val)
print('Validation accuracy:', accuracy_val)

Validation accuracy: 0.9529757531227039


In [66]:
accuracy_test = model.score(X_test, y_test)
print('Test accuracy:', accuracy_test)

Test accuracy: 0.9568944403624786


In [67]:
td=pd.read_csv("/kaggle/input/playground-series-s3e2/test.csv")
td

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown
1,15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown
2,15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown
3,15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes
4,15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes
...,...,...,...,...,...,...,...,...,...,...,...
10199,25503,Female,27.0,0,0,No,Private,Urban,75.77,17.6,never smoked
10200,25504,Male,49.0,0,0,Yes,Private,Urban,102.91,26.7,Unknown
10201,25505,Female,3.0,0,0,No,children,Rural,104.04,18.3,Unknown
10202,25506,Male,31.0,0,0,Yes,Private,Urban,82.41,28.7,never smoked


In [68]:
td.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64

In [69]:
td['work_type'] = td['work_type'].replace(work_mapping)
td['gender'] = td['gender'].replace(gender_mapping)
td['ever_married'] = td['ever_married'].replace(marriage_mapping)
td['Residence_type'] = td['Residence_type'].replace(residence_mapping)
td['smoking_status'] = td['smoking_status'].replace(smoke_mapping)
td

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,15304,1,57.0,0,0,1,3,0,82.54,33.4,3
1,15305,0,70.0,1,0,1,3,1,72.06,28.5,3
2,15306,1,5.0,0,0,0,0,1,103.72,19.5,3
3,15307,1,56.0,0,0,1,1,1,69.24,41.4,2
4,15308,0,32.0,0,0,1,3,0,111.15,30.1,2
...,...,...,...,...,...,...,...,...,...,...,...
10199,25503,1,27.0,0,0,0,3,1,75.77,17.6,1
10200,25504,0,49.0,0,0,1,3,1,102.91,26.7,3
10201,25505,1,3.0,0,0,0,0,0,104.04,18.3,3
10202,25506,0,31.0,0,0,1,3,1,82.41,28.7,1


In [70]:
sbms = td
sbms = sbms[['id']]

In [71]:
sbms = sbms.assign(stroke=None)
sbms

,id,stroke
0,15304,None
1,15305,None
2,15306,None
3,15307,None
4,15308,None
...,...,...
10199,25503,None
10200,25504,None
10201,25505,None
10202,25506,None


In [72]:
td.drop('id', axis=1, inplace=True)
td

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,1,57.0,0,0,1,3,0,82.54,33.4,3
1,0,70.0,1,0,1,3,1,72.06,28.5,3
2,1,5.0,0,0,0,0,1,103.72,19.5,3
3,1,56.0,0,0,1,1,1,69.24,41.4,2
4,0,32.0,0,0,1,3,0,111.15,30.1,2
...,...,...,...,...,...,...,...,...,...,...
10199,1,27.0,0,0,0,3,1,75.77,17.6,1
10200,0,49.0,0,0,1,3,1,102.91,26.7,3
10201,1,3.0,0,0,0,0,0,104.04,18.3,3
10202,0,31.0,0,0,1,3,1,82.41,28.7,1


In [73]:
sbms['stroke'] = model.predict_proba(td)
sbms

,id,stroke
0,15304,1.00
1,15305,0.70
2,15306,1.00
3,15307,0.83
4,15308,0.97
...,...,...
10199,25503,1.00
10200,25504,1.00
10201,25505,1.00
10202,25506,0.99


In [74]:
sbms[['id', 'stroke']].to_csv('/kaggle/working/output.csv', index=False)